### [Soynlp] 학습 기반 토크나이저 <hr>
- 품사 태깅, 단어 토큰화 등을 지원하는 단어 토크나이저
- 비지도 학습으로 단어 토큰화 ==> 데이터에 자주 등장하는 단어들을 단어로 분석
- 내부적으로 단어 점수 표로 동작

In [1]:
# !pip install soynlp

- 기존 형태소 분석기는 신조어나 형태소 분석기에 등록되지 않은 단어는 제대로 구분하지 못함

In [2]:
from konlpy.tag import Okt

tokenizer = Okt()

print(tokenizer.morphs('에이비식스 이대휘 1월 최애돌 기부 요정 입니다'))

# 형태소 분석시 매개변수 stem = True 설정
print(tokenizer.morphs('에이비식스 이대휘 1우러 최애돌 기부 요정 입니다', stem = True))

print(tokenizer.morphs('에이비식스 이대휘 1우러 최애돌 기부 요정 입니다', norm = True))

['에이', '비식스', '이대', '휘', '1월', '최애', '돌', '기부', '요정', '입니다']
['에이', '비식스', '이대', '휘', '1', '우러', '최애', '돌', '기부', '요정', '이다']
['에이', '비식스', '이대', '휘', '1', '우러', '최애', '돌', '기부', '요정', '입니다']


- [Sonlpy] 사용 => 말뭉치 데이터셋으로 학습 진행 후 사용 가능

In [3]:
file_name = '../DATA/text_data.txt'

In [4]:
# 학습 데이터 처리
from soynlp import DoublespaceLineCorpus  # 한개로 통합된 문서 데이터 분리
from soynlp.word import WordExtractor  # 단어 추출

In [5]:
# 훈련 데이터 문서 분리
corpus = DoublespaceLineCorpus(file_name)
print(f'훈련 데이터 문서 : {len(corpus)}개')

훈련 데이터 문서 : 30091개


In [6]:
# Sonlpy 학습 진행
word_extractor = WordExtractor()
# 학습 진행하며
word_extractor.train(corpus)
# 단어별 점수표 추출
word_score_table = word_extractor.extract()

training was done. used memory 1.080 Gb
all cohesion probabilities was computed. # words = 223348
all branching entropies was computed # words = 361598
all accessor variety was computed # words = 361598


In [7]:
# 단어별 점수표 확인
for idx, key in enumerate(word_score_table.keys()):
    print(f'[{idx}] = {key}')
    if idx == 30: break

[0] = 카
[1] = 핏
[2] = 뾰
[3] = 뭡
[4] = 캉
[5] = 혼
[6] = 삭
[7] = 왓
[8] = 펴
[9] = 셋
[10] = 뚜
[11] = 박
[12] = 2
[13] = 쉴
[14] = 움
[15] = 뛴
[16] = 우
[17] = 얼
[18] = 투
[19] = 찜
[20] = 막
[21] = 뒤
[22] = 윙
[23] = 렛
[24] = 지
[25] = 넴
[26] = 갉
[27] = 함
[28] = 김
[29] = 휴
[30] = 붙


- 응집 확률(cohesion probability) : 내부 문자열(substring)이 얼마나 응집하여 자주 등장하는지를 판단하는 척도  
==> 원리 : 문자열을 문자 단위로 분리, 왼쪽부터 순서대로 문자를 추가  
==> 각 문자열이 주어졌을 때 그 다음 문자가 나올 확률을 계산/ 누적곱 한 값  
==> 값이 높을수록 전체 코퍼스에서 이 문자열 시퀀스는 하나의 단어로 등장할 가능성 높음

In [8]:
word_score_table['바다'].cohesion_forward

0.06393648140409527

In [9]:
word_score_table['바다에'].cohesion_forward

0.11518621707955429

In [10]:
word_score_table['바다를'].cohesion_forward

0.07716779358040307

- SOYNLP의 L tokenzier  
==> 띄어쓰기 단위로 나눈 어절 토큰 : L토큰 + R토큰  
==> 예 : '공원에' => '공원' + '에', '공부하는' => '공부' + '하는'  
==> 분리 기준 : 점수가 가장 높은 L토큰을 찾아내는 원리

In [13]:
from soynlp.tokenizer import LTokenizer

# 토큰으로 쪼개기위한 L토큰 기준값
scores = {word : score.cohesion_forward for word, score in word_score_table.items()}

l_tokernizer = LTokenizer(scores = scores)
l_tokernizer.tokenize('국제사회와 우리의 노력들로 범죄를 척결하자', flatten = False)

[('국제사회', '와'), ('우리', '의'), ('노력', '들로'), ('범죄', '를'), ('척결', '하자')]

- 최대 점수 토크나이저  
==> 띄어쓰기가 되어 있지 않은 문장에서 점수가 높은 글자 시퀀스를 순차적으로 찾아내는 토크나이저  
==> 띄어쓰기가 되어 있지 않은 문장을 넣어서 점수를 통해 토큰화 된 결과

In [14]:
from soynlp.tokenizer import MaxScoreTokenizer

maxscore_tokenizer = MaxScoreTokenizer(scores = scores)
maxscore_tokenizer.tokenize('국제사회와우리의노력들로범죄를척결하자')

['국제사회', '와', '우리', '의', '노력', '들로', '범죄', '를', '척결', '하자']

In [15]:
maxscore_tokenizer('국제사회와 우리의 노력들로 범죄를 척결하자')

['국제사회', '와', '우리', '의', '노력', '들로', '범죄', '를', '척결', '하자']

In [16]:
word_score_table['국'].cohesion_forward, word_score_table['국제'].cohesion_forward

(0, 0.07856876882976202)

In [17]:
word_score_table['국제사'].cohesion_forward, word_score_table['국제사회'].cohesion_forward

(0.09217735975351507, 0.20075093164820865)

SOYNLP를 이용한 반복되는 문자 정제  
==> ㅋㅋ, ㅎㅎ 등의 이모티콘의 경우 불필요하게 연속되는 경우 많음  
==> 반복되는 것은 하나로 정규화

In [18]:
from soynlp.normalizer import *
import string
print(string.punctuation)

print(emoticon_normalize('앜^^^^^^^', num_repeats = 1))
print(emoticon_normalize('앜ㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠ', num_repeats = 1))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠ', num_repeats = 2))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠㅠ', num_repeats = 2))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠㅠㅠㅠ', num_repeats = 2))

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
앜^^^^^^^
아ㅋ영화존잼쓰ㅠ
아ㅋㅋ영화존잼쓰ㅠㅠ
아ㅋㅋ영화존잼쓰ㅠㅠ
아ㅋㅋ영화존잼쓰ㅠㅠ


In [ ]:
# !pip install Customized_konlpy

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/881.5 kB ? eta -:--:--
   ---------------------------------------- 10.2/881.5 kB ? eta -:--:--
   ----- ---------------------------------- 112.6/881.5 kB 1.7 MB/s eta 0:00:01
   -------------------------- ------------- 573.4/881.5 kB 5.2 MB/s eta 0:00:01
   ---------------------------------------- 881.5/881.5 kB 7.0 MB/s eta 0:00:00


In [19]:
from ckonlpy.tag import Twitter

twitter = Twitter()
twitter.morphs('은경이는 사무실로 갔습니다.') 

c:\ProgramData\anaconda3\envs\Torch_NLP38\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


['은', '경이', '는', '사무실', '로', '갔습니다', '.']

In [20]:
# 형태소 분석기에 사전 추가
twitter.add_dictionary('은경이', 'Noun')